In [10]:
import cv2
import mediapipe as mp
import pyautogui

In [11]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
cap = cv2.VideoCapture(0)

In [12]:
def get_hand_landmarks(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        return result.multi_hand_landmarks[0]
    return None

def get_finger_pos(landmarks, width, height):
    finger_pos = []
    for id, lm in enumerate(landmarks.landmark):
        x,y = int(lm.x*width), int(lm.y*height)
        finger_pos.append((x,y))
    return finger_pos

def move_mouse(index_finger_pos):
    screen_width, screen_height = pyautogui.size()
    mouse_x = int(index_finger_pos[0]*screen_width/width)
    mouse_y = int(index_finger_pos[1]*screen_height/height)
    pyautogui.moveTo(mouse_x, mouse_y)
    
def click_mouse():
    pyautogui.click()

In [13]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    height, width, _ = frame.shape
    landmarks = get_hand_landmarks(frame)
    if landmarks:
        finger_pos = get_finger_pos(landmarks, width, height)
        index_finger_pos = finger_pos[8]
        thumb_tip_pos = finger_pos[4]
        move_mouse(index_finger_pos)
        if abs(index_finger_pos[0]-thumb_tip_pos[0])<10 and abs(index_finger_pos[1] - thumb_tip_pos[1])<10:
            click_mouse()
        mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imshow("Virtual Mouse", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()